# Surrogate models for climate metrics

The purpose of this notebook is to provide regression models to calculate climate metrics as functions of time horizon and aviation species settings (sensitivity to emissions, ERF/RF and efficacy):
$$ Metric = f(H, \sigma, \eta, r) $$

The regression models are fitted here on FaIR's outputs.

In [2]:
# Import libraries
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import qmc
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Climate functions
from aerometrics.utils.functions import emission_profile_function
from aerometrics.metrics.metrics import co2_ipcc_pulse_absolute_metrics, absolute_metrics, relative_metrics
from aerometrics.climate_models.co2_ipcc_climate_model import CO2_IPCC_ClimateModel
from aerometrics.climate_models.gwpstar_climate_model import GWPStarClimateModel
from aerometrics.climate_models.lwe_climate_model import LWEClimateModel
from aerometrics.climate_models.fair_climate_model import FairClimateModel

## Sampling

In [3]:
# Define variable bounds (in log-space)
variables = {"time_horizon_log": (np.log(10), np.log(200))}

# Generate Latin Hypercube samples
n_samples = 100
sampler = qmc.LatinHypercube(d=len(variables))
sample = sampler.random(n=n_samples)  # scaled between 0 and 1

# Rescale samples to variable bounds
l_bounds, u_bounds = zip(*variables.values())
scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

# Round to integer years in original space, then convert back to log --> ensure further processing with time horizons will use int years
scaled_sample = np.log(np.exp(scaled_sample).astype(int))
df_samples = pd.DataFrame(scaled_sample, columns=variables.keys())
df_samples

## Calculate metrics

In [6]:
# Species settings
SPECIES_SETTINGS = {
    "Contrails": {
        "sensitivity_rf": 2.23e-12,
        "ratio_erf_rf": 0.42,
        "efficacy_erf": 1.0,
    },
    "NOx - ST O3 increase": {
        "sensitivity_rf": 25.1e-12 * (14 / 46),
        "ratio_erf_rf": 1.37,
        "efficacy_erf": 1.0,
    },
    "NOx - CH4 decrease and induced": {
        "ch4_loss_per_nox": -3.90,
        "ratio_erf_rf": 1.18,
        "efficacy_erf": 1.0,
    },
}
unit_value = 1*10**10

# CO2 settings
species_settings_co2 = {
    #"sensitivity_rf": 9.2e-16, 
    "ratio_erf_rf": 1.0, 
    "efficacy_erf": 1.0
}
unit_value_co2 = 1*10**10

# model settings for FaIR
model_settings = {"rcp": "RCP45"}

# Time settings
start_year = 1765
t0 = 2020

# repare collection for results
all_results = []

# Loop over each species
for species, species_settings in SPECIES_SETTINGS.items():
    print(f"\n Computing metrics for {species}...")

    species_results = []
    
    # Loop over each time horizon sample
    for i, log_horizon in tqdm(enumerate(df_samples["time_horizon_log"]), total=n_samples, leave=False):
        time_horizon = int(np.exp(log_horizon))  # integer time horizons are required by aerometrics functions
        end_year = t0 + time_horizon

        # CO2 absolute metrics
        emission_profile_co2 = emission_profile_function(start_year, t0, time_horizon, "pulse", unit_value_co2)
        results_co2 = FairClimateModel(
            start_year,
            end_year,
            "CO2",
            emission_profile_co2,
            species_settings_co2,
            model_settings
        ).run()
        rf_co2_u = results_co2["radiative_forcing"] / unit_value_co2
        erf_co2_u = results_co2["effective_radiative_forcing"] / unit_value_co2
        temp_co2_u = results_co2["temperature"] / unit_value_co2

        agwp_rf_co2, agwp_erf_co2, aegwp_rf_co2, aegwp_erf_co2, agtp_co2, iagtp_co2, atr_co2 = absolute_metrics(
            rf_co2_u, 
            erf_co2_u, 
            species_settings_co2.get("efficacy_erf", 1.0), 
            temp_co2_u, 
            time_horizon
        )

        # Species absolute metrics
        emission_profile = emission_profile_function(start_year, t0, time_horizon, "pulse", unit_value)
        results = FairClimateModel(
            start_year, 
            end_year, 
            species, 
            emission_profile, 
            species_settings, 
            model_settings
        ).run()
        rf_u = results["radiative_forcing"] / unit_value
        erf_u = results["effective_radiative_forcing"] / unit_value
        temp_u = results["temperature"] / unit_value

        agwp_rf, agwp_erf, aegwp_rf, aegwp_erf, agtp, iagtp, atr = absolute_metrics(
            rf_u, erf_u, species_settings.get("efficacy_erf", 1.0), temp_u, time_horizon
        )

        # Relative metrics
        gwp_rf, gwp_erf, egwp_rf, egwp_erf, gtp, igtp, ratr = relative_metrics(
            agwp_rf_co2, agwp_erf_co2, aegwp_rf_co2, aegwp_erf_co2,
            agtp_co2, iagtp_co2, atr_co2,
            agwp_rf, agwp_erf, aegwp_rf, aegwp_erf, agtp, iagtp, atr
        )

        species_results.append({
            "species": species,
            "time_horizon": time_horizon,
            "GWP_RF": gwp_rf,
            "GWP_ERF": gwp_erf,
            "EGWP_RF": egwp_rf,
            "EGWP_ERF": egwp_erf,
            "GTP": gtp,
            "iGTP": igtp,
            "RATR": ratr,
        })

    all_results.extend(species_results)

# Combine results for all species
df = pd.DataFrame(all_results)
print(df.head())

In [13]:
df

## Fit regression models

### Model 1 (power law)
$$ \log(\text{metric}) = a_1 + b_1 \log(\text{time horizon}) $$
$$ \text{metric} = e^{a_1} \text{time horizon}^{b_1} $$

### Model 2 (log-quadratic model)
$$ \log(\text{metric}) = a_2 + b_2 \log(\text{time horizon}) + c_2 [\log(\text{time horizon})]^2 $$
$$ \text{metric} = e^{a_2} \text{time horizon}^{b_2 + c_2 \log(\text{time horizon})} $$

In [14]:
def fit_log_models(df, species, metric):
    """
    Fit two regression models for a given species and metric:
      - Model 1: linear in log(feature)
      - Model 2: quadratic in log(feature)
    
    Args:
        df (pd.DataFrame): DataFrame containing columns ['species', 'time_horizon', metric]
        species (str): species name to filter the data
        metric (str): column name of the metric to fit
        
    Returns:
        dict: containing Model1, Model2, analytical expressions, and errors
    """
    # Filter data for species
    df_s = df[df["species"] == species].copy()
    if df_s.empty:
        raise ValueError(f"No data found for species '{species}'")
    
    # Prepare features
    X = df_s[["time_horizon"]].copy()
    X["time_horizon_log"] = np.log(X["time_horizon"])
    X["time_horizon_log_2"] = X["time_horizon_log"] ** 2
    y = np.log(df_s[metric]) if species != "NOx - CH4 decrease and induced" else np.log(-df_s[metric])
    
    # Model 1: linear in log(time_horizon)
    model1 = LinearRegression().fit(X[["time_horizon_log"]], y)
    a1 = model1.intercept_
    b1 = model1.coef_[0]
    eq1_log = f"log({metric}) = {a1:.4f} + {b1:.4f} × log(time_horizon)"
    eq1 = f"{metric} = {np.exp(a1):.4f} × time_horizon^{b1:.4f}"
    y_pred1 = model1.predict(X[["time_horizon_log"]])
    r2_1 = r2_score(np.exp(y), np.exp(y_pred1))
    
    # Model 2: quadratic in log(time_horizon)
    model2 = LinearRegression().fit(X[["time_horizon_log", "time_horizon_log_2"]], y)
    a2 = model2.intercept_
    b2, c2 = model2.coef_
    eq2_log = f"log({metric}) = {a2:.4f} + {b2:.4f} × log(time_horizon) + {c2:.4f} × log(time_horizon)^2"
    eq2 = f"{metric} = {np.exp(a2):.4f} × time_horizon^({b2:.4f} + {c2:.4f} × log(time_horizon))"
    y_pred2 = model2.predict(X[["time_horizon_log", "time_horizon_log_2"]])
    r2_2 = r2_score(np.exp(y), np.exp(y_pred2))
    
    return {
        "Model1": model1,
        "Model2": model2,
        "eq1_log": eq1_log,
        "eq1": eq1,
        "eq2_log": eq2_log,
        "eq2": eq2,
        "R2_1": r2_1,
        "R2_2": r2_2,
        "df": pd.DataFrame({"y": np.exp(y), "y_pred1": np.exp(y_pred1), "y_pred2": np.exp(y_pred2)})
    }


def plot_surrogate(results, bin_width: float = 1.0):
    """
    Plot the results of the model fitting with an observed-estimated chart and an histogram of errors.
    """
    # Retrieve results
    y = results["df"]["y"]
    y_pred1 = results["df"]["y_pred1"]
    y_pred2 = results["df"]["y_pred2"]
    residuals_1 = (y_pred1 - y) / y * 100
    residuals_2 = (y_pred2 - y) / y * 100
    mean_error1 = np.mean(residuals_1)
    mean_error2 = np.mean(residuals_2)
    r21 = results['R2_1']
    r22 = results['R2_2']

    # Set figures
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))

    # Observed-estimated chart
    axes[0].scatter(y, y_pred1, label='Model 1 - '+f'R² = {r21:.3f}', alpha=0.7)
    axes[0].scatter(y, y_pred2, label='Model 2 - '+f'R² = {r22:.3f}', alpha=0.7)
    axes[0].plot([np.min(y), np.max(y)], [np.min(y), np.max(y)], color='black')
    axes[0].set_xlabel(r"Observed")
    axes[0].set_ylabel(r"Estimated")
    axes[0].legend()
    axes[0].grid(True)

    # Regression errors histogram
    combined_min = min(residuals_1.min().item(), residuals_2.min().item())
    combined_max = max(residuals_1.max().item(), residuals_2.max().item())
    bins = np.arange(combined_min, combined_max + bin_width, bin_width)
    axes[1].hist(residuals_1, bins=bins, color='C0', edgecolor='black', alpha=0.7)
    axes[1].hist(residuals_2, bins=bins, color='C1', edgecolor='black', alpha=0.7)
    axes[1].axvline(mean_error1, color='C0', linestyle='--', linewidth=2, label='Model 1 - '+f'Mean error: {mean_error1:.2f}')
    axes[1].axvline(mean_error2, color='C1', linestyle='--', linewidth=2, label='Model 2 - 'f'Mean error: {mean_error2:.2f}')
    axes[1].set_xlabel("Relative Error [%]")
    axes[1].set_ylabel("Frequency [%]")
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

    return fig, axes

## Applications

### r-ATR for NOx

In [15]:
# Set species and metric to estimate
species="NOx - CH4 decrease and induced"
metric="RATR"

# Get results
results = fit_log_models(df, species=species, metric=metric)
print(results["eq1"])
print(results["eq2"])
print(f"Model 1 R²: {results['R2_1']:.4f}")
print(f"Model 2 R²: {results['R2_2']:.4f}")
plot_surrogate(results)
#plt.savefig("figures/surrogate_model.pdf")

### EGWP for contrails

In [11]:
# Set species and metric to estimate
species="Contrails"
metric="EGWP_ERF"

# Get results
results = fit_log_models(df, species=species, metric=metric)
print(results["eq1"])
print(results["eq2"])
print(f"Model 1 R²: {results['R2_1']:.4f}")
print(f"Model 2 R²: {results['R2_2']:.4f}")
plot_surrogate(results, bin_width=0.3)
#plt.savefig("figures/surrogate_model.pdf")